In [11]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName('Analyzing the vocabulary of Pride and Prejudice')
         .getOrCreate())

In [12]:
import requests 
import tempfile

url = 'https://www.gutenberg.org/cache/epub/1342/pg1342.txt'
response = requests.get(url)
text = response.text

with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
    temp_file.write(text)
    temp_path = temp_file.name

book = spark.read.text(temp_path)



In [14]:
book.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
print(book.dtypes)

[('value', 'string')]


In [ ]:
# Show the first 25 rows and truncate the row to 80 characters

book.show(25, truncate=80)

+--------------------------------------------------------------------------------+
|                                                                           value|
+--------------------------------------------------------------------------------+
|                              The Project Gutenberg eBook of Pride and Prejudice|
|                                                                                |
|           This ebook is for the use of anyone anywhere in the United States and|
|        most other parts of the world at no cost and with almost no restrictions|
|          whatsoever. You may copy it, give it away or re-use it under the terms|
|             of the Project Gutenberg License included with this ebook or online|
|              at www.gutenberg.org. If you are not located in the United States,|
|            you will have to check the laws of the country where you are located|
|                                                        before using this eBook.|
|   